In [1]:
from scnn.bipolar_functions import *
from scnn.convertors import F2S, S2F
import torch

In [2]:
f2s = F2S(seq_len=10000, mode='bipolar')
s2f = S2F('bipolar')

In [23]:
x = f2s.generate(torch.rand(2,10))

In [13]:
y = f2s.generate(torch.tensor(-0.3))

In [14]:
x.shape

torch.Size([10000])

In [15]:
add = scaled_add()

In [16]:
out = add(x,y,None)

In [17]:
s2f.evaluate(out)

tensor(0.0902)

In [18]:
out.shape

torch.Size([10000])

In [19]:
out.sum()

tensor(5451)

In [2]:
x = torch.rand(3,10)

In [4]:

torch.full(x.shape[:-2], 2)

tensor(2)

In [10]:
x.shape

torch.Size([2, 3, 10000])

In [11]:
torch.sum(x,dim=-2).shape

torch.Size([2, 10000])

In [12]:
x.shape

torch.Size([2, 3, 10000])

In [14]:
x.shape

torch.Size([2, 3, 10000])

In [15]:
mask = x==1

In [16]:
x[mask]=0

In [18]:
x.shape

torch.Size([2, 3, 10000])

In [20]:
x.dim()

3

In [86]:
def counter(x):
    return torch.sum(x, dim=-2, dtype=torch.int16)
def neuron_c(inputs, r: int):
    num_in, seq_len = inputs.shape[-2], inputs.shape[-1]
    s_max = r-1
    s_half = (r-1)/2.

    s = torch.full(inputs.shape[:-2], s_half, dtype=torch.int16)
    v = counter(inputs)*2 - num_in
    out = torch.zeros(inputs.shape[:-2]+(seq_len,), dtype=torch.bool)

    for i in range(seq_len):
        s = s + v.select(dim=-1, index=i)
        overflow_idx = s > s_max
        underflow_idx = s < 0
        s[overflow_idx] = s_max
        s[underflow_idx] = 0
        
        out[..., i] = s > s_half
    return out

In [96]:
x = f2s.generate(torch.rand(100)*2-1)
x_real = s2f.evaluate(x)

In [97]:
x.shape

torch.Size([100, 10000])

In [98]:
s2f.evaluate(neuron_c(x,r=2))

tensor(0.8084)

In [99]:
import numpy as np
np.tanh(0.10*x_real.sum(dim=-1))

tensor(0.7613)

In [32]:
x_real = s2f.evaluate(x)

In [33]:
x_real

tensor([[0.4614, 0.6720, 0.1362, 0.4000, 0.7102, 0.3472, 0.5478, 0.6186, 0.1210,
         0.3720],
        [0.2442, 0.9326, 0.6950, 0.2208, 0.2060, 0.7914, 0.8724, 0.1700, 0.1674,
         0.4180]])

In [145]:
x = f2s.generate(torch.rand(256)*2-1)

In [1]:
import torch
from scnn.bipolar_functions.operations import *
from scnn.convertors import F2S
from scnn.convertors import F2S, S2F


class MUXScaled_Linear():
    def __init__(self, 
                 in_features,
                 out_features,
                 seq_len,
                 bias=False) -> None:
        super().__init__()

        self.down_scale = in_features
        self.in_features = in_features
        self.out_features = out_features
        self.seq_len = seq_len
        self.is_bias = bias
        self.weight = None
        self.bias = None

    def load_weight(self, data):
        assert data['weight'].shape == torch.Size([self.in_features, self.out_features])

        self.weight = F2S(seq_len=self.seq_len)(data['weight'])

        if self.is_bias:
            assert data['bias'].shape == torch.Size([self.out_features])
            self.bias = F2S(seq_len=self.seq_len)(data['bias']/self.in_features)

    def forward(self, inputs):
        assert self.weight is not None
        out = scaled_matmul(inputs, self.weight)

        out = scaled_add(out, self.bias)

        return out


class APCLinear():
    def __init__(self, 
                 in_features,
                 out_features,
                 seq_len,
                 btanh_scalar,
                 bias=False) -> None:
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.seq_len = seq_len
        self.is_bias = bias
        self.weight = None
        self.bias = None
        self.btanh_scalar =  btanh_scalar
        self.states = self.get_btanh_states()
        
    def load_weight(self, data):
        assert data['weight'].shape == torch.Size([self.in_features, self.out_features])

        self.weight = F2S(seq_len=self.seq_len)(data['weight'])

        if self.is_bias:
            assert data['bias'].shape == torch.Size([self.out_features])
            self.bias = F2S(seq_len=self.seq_len)(data['bias']/self.in_features)
    def get_btanh_states(self,):
        q = 1.835*(2*self.in_features)**(-0.5552)
        r_prime = 2*self.in_features + 2*(self.btanh_scalar-1)*(self.in_features-1)/(1-q)
        return int(round(r_prime/2.)*2)

    
    def forward(self, inputs):
        out = torch.zeros(inputs.shape[:-2] + (self.out_features,self.seq_len,))

        for i in range(self.out_features):
            out[...,i,:] = neuron_c(mul(inputs, self.weight[:,i]), self.states)

        if self.bias:
            out = scaled_add(out,self.bias)
            
        return out


In [56]:
model1 = APCLinear(in_features=100, out_features=10, seq_len=16384*4, btanh_scalar=1)

In [57]:
weight = torch.rand(100,10)*2-1

In [58]:
model1.load_weight(data={'weight':weight})

In [59]:
inputs = (torch.rand(2,3,100)*2-1)/4

In [60]:
inputs_sc = F2S(16384*4).generate(inputs)

In [61]:
inputs_sc.shape

torch.Size([2, 3, 100, 65536])

In [62]:
out_sc = model1.forward(inputs_sc)

In [63]:
S2F().evaluate(out_sc)

tensor([[[ 0.7106,  0.6154, -0.1577, -0.1710,  0.3834,  0.2123, -0.5586,
          -0.4219, -0.4800,  0.4801],
         [ 0.2551, -0.6721,  0.7400, -0.3076, -0.5714, -0.0507,  0.5511,
          -0.5118, -0.2481,  0.3784],
         [-0.4697, -0.4018,  0.4093,  0.7182, -0.3066,  0.1433,  0.4494,
           0.7934,  0.6938,  0.8160]],

        [[-0.1683,  0.7631,  0.4793,  0.7556,  0.3472, -0.6100, -0.2180,
           0.6491,  0.5699,  0.7145],
         [-0.0338,  0.6920, -0.4961,  0.7137, -0.7158, -0.2650,  0.7094,
           0.5712,  0.0967, -0.3492],
         [-0.6051,  0.6844, -0.6874,  0.1285,  0.3412, -0.2303,  0.1494,
          -0.5961, -0.2078, -0.3615]]])

In [64]:
out = torch.tanh(inputs @ weight)
print(out)

tensor([[[ 0.8361,  0.7925, -0.2461, -0.2287,  0.5458,  0.3379, -0.6446,
          -0.6053, -0.5767,  0.6064],
         [ 0.3385, -0.8119,  0.9025, -0.4361, -0.7331, -0.0572,  0.6746,
          -0.6463, -0.3040,  0.4721],
         [-0.7006, -0.6124,  0.5567,  0.8145, -0.3914,  0.2598,  0.6097,
           0.9107,  0.8507,  0.9293]],

        [[-0.2821,  0.8987,  0.5870,  0.8751,  0.4560, -0.7826, -0.2607,
           0.8181,  0.7911,  0.8337],
         [ 0.0424,  0.8590, -0.6890,  0.8801, -0.8859, -0.2709,  0.8446,
           0.7114,  0.1187, -0.4721],
         [-0.7831,  0.8326, -0.8410,  0.2229,  0.4600, -0.4762,  0.1551,
          -0.6538, -0.1774, -0.5493]]])


In [107]:
inputs = (torch.rand(768)*2-1)/8
inputs_sc = F2S(16).generate(inputs)

In [104]:
apc = ap_counter(in_features=768, num_au_layers=1)

In [105]:
apc(inputs_sc)

tensor([414, 382, 422, 386, 384, 392, 404, 382, 394, 372, 410, 372, 370, 372,
        398, 410], dtype=torch.int16)

In [106]:
inputs_sc.sum(dim=-2)

tensor([406, 385, 406, 387, 391, 386, 413, 380, 383, 378, 411, 386, 378, 369,
        388, 395])

In [79]:
S2F()(F2S(16384*4)(inputs)).sum()

tensor(1.6205)

In [88]:
S2F()(neuron_c(F2S(16384*4)(inputs), 256))

tensor(0.4356)

In [50]:
def get_btanh_scalar(in_features, num_states):
    q = 1.835*(2*in_features)**(-0.5552)
    return (1-q)/(2*(in_features-1))*(num_states-2*in_features)+1

def get_btanh_states(in_features, scalar):
    q = 1.835*(2*in_features)**(-0.5552)
    r_prime = 2*in_features + 2*(scalar-1)*(in_features-1)/(1-q)
    return int(round(r_prime/2.)*2)

get_btanh_scalar(768, 256)

0.1916378942826471